# Introduction to modules, layers, and models

## 1. Setup

In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

print("The version of Tensorflow: {}".format(tf.__version__))

The version of Tensorflow: 2.4.1


## 2. Defining models and layers in TensorFlow

In [2]:
# Here's an example of a very simple tf.Module that operates on a scalar tensor:

class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [3]:
# By subclassing tf.Module, any tf.Variable or tf.Module instances assigned to 
# this object's properties are automatically collected. 
# This allows you to save and load variables, and also create collections of tf.Modules.

# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


In [4]:
# This is an example of a two-layer linear layer model made out of modules.

# First a dense (linear) layer:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [5]:
# And then the complete model, which makes two layer instances and applies them:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.        6.6957006]], shape=(1, 2), dtype=float32)


In [6]:
# tf.Module instances will automatically collect, recursively, any tf.Variable or tf.Module instances assigned to it. 
# This allows you to manage collections of tf.Modules with a single model instance, and save and load whole models.

print("Submodules:", my_model.submodules)

Submodules: (<__main__.Dense object at 0x000001F72D015190>, <__main__.Dense object at 0x000001F6CD840130>)


In [7]:
for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 1.4872581 , -0.67542785,  0.7220626 ],
       [-0.1911573 , -1.0964158 ,  2.7435508 ],
       [ 0.48312464,  0.85014474, -0.13067296]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.64639187,  0.13822186],
       [ 0.35022366,  0.3519714 ],
       [-0.3433067 ,  0.9301283 ]], dtype=float32)> 



### i. Waiting to create variables

In [8]:
class FlexibleDenseModule(tf.Module):
  # Note: No need for `in_features`
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    # Create variables on first call.
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [9]:
# Used in a module
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[3.1787279 1.0092701]], shape=(1, 2), dtype=float32)


## 3. Saving weights

In [10]:
# You can save a tf.Module as both a checkpoint and a SavedModel.

# Checkpoints are just the weights (that is, the values of the set of variables inside the module and its submodules):

chkp_path = "my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'my_checkpoint'

In [11]:
%ls my_checkpoint*

 驱动器 D 中的卷是 Data
 卷的序列号是 12E0-D62D

 D:\miniconda\workspace\TensorFlow2Gride 的目录

2021/04/04  17:18               427 my_checkpoint.data-00000-of-00001
2021/04/04  17:18               362 my_checkpoint.index
               2 个文件            789 字节
               0 个目录 318,344,138,752 可用字节


In [12]:
# You can look inside a checkpoint to be sure the whole collection of variables is saved, 
# sorted by the Python object that contains them.

tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [13]:
# When you load models back in, you overwrite the values in your Python object.

new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("my_checkpoint")

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.1787279, 1.0092701]], dtype=float32)>

## 4. Saving functions

In [14]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

In [15]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0. 0.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)


In [16]:
# You can visualize the graph by tracing it within a TensorBoard summary.

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([[0.        2.0163946]], shape=(1, 2), dtype=float32)


In [18]:
# Launch TensorBoard to view the resulting trace:
%tensorboard --logdir logs/func

Reusing TensorBoard on port 6006 (pid 10340), started 0:16:35 ago. (Use '!kill 10340' to kill it.)

### i. Creating a SavedModel

In [20]:
# You can save the model you have just trained as follows:
tf.saved_model.save(my_model, "the_saved_model")

INFO:tensorflow:Assets written to: the_saved_model\assets


In [25]:
# Inspect the SavedModel in the directory
%ls the_saved_model

 驱动器 D 中的卷是 Data
 卷的序列号是 12E0-D62D

 D:\miniconda\workspace\TensorFlow2Gride\the_saved_model 的目录

2021/04/04  17:21    <DIR>          .
2021/04/04  17:21    <DIR>          ..
2021/04/04  17:21    <DIR>          assets
2021/04/04  17:21            14,140 saved_model.pb
2021/04/04  17:21    <DIR>          variables
               1 个文件         14,140 字节
               4 个目录 318,344,073,216 可用字节


In [29]:
# The variables/ directory contains a checkpoint of the variables
%ls the_saved_model\variables

 驱动器 D 中的卷是 Data
 卷的序列号是 12E0-D62D

 D:\miniconda\workspace\TensorFlow2Gride\the_saved_model\variables 的目录

2021/04/04  17:21    <DIR>          .
2021/04/04  17:21    <DIR>          ..
2021/04/04  17:21               408 variables.data-00000-of-00001
2021/04/04  17:21               356 variables.index
               2 个文件            764 字节
               2 个目录 318,344,073,216 可用字节


In [30]:
# You can load the model as new object:

new_model = tf.saved_model.load("the_saved_model")

In [31]:
# new_model, created from loading a saved model, is an internal TensorFlow user object without any of the class knowledge. 
# It is not of type SequentialModule.

isinstance(new_model, SequentialModule)

False

In [32]:
# This new model works on the already-defined input signatures. You can't add more signatures to a model restored like this.

print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0. 0.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)


## 5. Keras models and layers

### i. Keras layers

In [33]:
# tf.keras.layers.Layer is the base class of all Keras layers, and it inherits from tf.Module.

# You can convert a module into a Keras layer just by swapping out the parent and then changing __call__ to call:

class MyDense(tf.keras.layers.Layer):
  # Adding **kwargs to support base Keras layer arguments
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    # This will soon move to the build step; see below
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

In [34]:
simple_layer([[2.0, 2.0, 2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.       , 3.3632455, 1.8337829]], dtype=float32)>

### ii. The build step

In [35]:
# You can rewrite MyDense layer above to be flexible to the size of its inputs:

class FlexibleDense(tf.keras.layers.Layer):
  # Note the added `**kwargs`, as Keras supports many arguments
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  # Create the state of the layer (weights)
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  # Defines the computation from inputs to outputs
    return tf.matmul(inputs, self.w) + self.b

# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

In [36]:
# At this point, the model has not been built, so there are no variables:

flexible_dense.variables

[]

In [37]:
# Calling the function allocates appropriately-sized variables:

# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[1.4723215 2.3819118 2.5421505]
 [2.2084823 3.5728676 3.8132257]], shape=(2, 3), dtype=float32)


In [38]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.1288026 ,  1.0187075 ,  0.28111425],
        [ 0.39145884,  1.373036  ,  0.93163   ],
        [ 0.47350454, -1.2007878 ,  0.058331  ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [39]:
# Since build is only called once, inputs will be rejected if the input shape is not compatible with the layer's variables:

try:
  print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
  print("Failed:", e)

Failed: Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]


### iii. Keras models

In [40]:
# You can define the SequentialModule from above with nearly identical code, 
# again converting __call__ to call() and changing the parent:

class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.13101035 1.0429598 ]], shape=(1, 2), dtype=float32)


In [41]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.5149364 , -1.0164068 ,  0.6384996 ],
        [ 0.37633342,  0.20298098,  0.04507061],
        [-0.8428199 ,  1.3251464 , -0.7946522 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-1.0019482,  1.2314947],
        [ 0.1557269,  0.784778 ],
        [-0.309328 , -0.5421853]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [42]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x1f7332f3580>,
 <__main__.FlexibleDense at 0x1f7337a9850>)

In [43]:
# Here is the same model with the functional API:

inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
flexible_dense_3 (FlexibleDe (None, 3)                 12        
_________________________________________________________________
flexible_dense_4 (FlexibleDe (None, 2)                 8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [44]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.0714188, -1.7540443]], dtype=float32)>

## 6. Saving Keras models

In [45]:
# Keras models can be checkpointed, and that will look the same as tf.Module.

# Keras models can also be saved with tf.saved_models.save(), as they are modules. 
# However, Keras models have convenience methods and other functionality:

my_sequential_model.save("exname_of_file")

INFO:tensorflow:Assets written to: exname_of_file\assets


In [46]:
# Just as easily, they can be loaded back in:
reconstructed_model = tf.keras.models.load_model("exname_of_file")

In [47]:
# Keras SavedModels also save metric, loss, and optimizer states.

# This reconstructed model can be used and will produce the same result when called on the same data:

reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.13101035, 1.0429598 ]], dtype=float32)>

# reference

### https://tensorflow.google.cn/guide/intro_to_modules